<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [0]:
import json
from pathlib import Path
from glob import glob
import pandas as pd
import os

os.chdir('/content/drive/My Drive/idc/nlp/ex1/')

def preprocess():
  vocabulary = set()
  pathlist = glob('*.csv')
  for data_file_path in pathlist:
      data = pd.read_csv(data_file_path)
      for text in data['tweet_text'].values:
          vocabulary.update(list(text))

  # vocabulary.discard(' ')
  # vocabulary.discard('\t')
  # vocabulary.discard('\r')
  # vocabulary.discard('\n')

  return sorted(list(vocabulary))

  

In [3]:
vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [0]:
from collections import defaultdict
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  voc_size = len(vocabulary)
  if add_one:
      model = defaultdict(lambda: defaultdict(lambda: 1/voc_size))
  else:
      model = defaultdict(lambda: defaultdict(lambda: 1e-08))
  data_file = pd.read_csv(data_file_path)
  for data in data_file['tweet_text'].values:
      data = ["<s>"] + list(data) + ["</s>"]
      for i in range(len(data) - n + 1):
          word, char = ''.join(data[i:i + n - 1]), data[i + n - 1]
          model[''.join(word)][char] = model[word].get(char, 0) + 1

  for word, counts in model.items():
      if add_one:
          total_count = float(sum(counts.values())) + voc_size
          for char in counts:
              counts[char] = (counts[char] + 1) / total_count
      else:
          total_count = float(sum(counts.values()))
          for char in counts:
              counts[char] /= total_count

  return model



In [23]:
model = lm(3, vocabulary, 'en.csv', False)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
print(sum(v for v in model['ab'].values()))
model['ab']


{'<s>R': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7fe41f5d8400>, {'T': 0.9881792183031458, 'o': 0.001143946615824595, 'e': 0.006101048617731172, 'y': 0.00038131553860819826, 'i': 0.001143946615824595, '.': 0.00019065776930409913, 'E': 0.0005719733079122975, 'u': 0.0005719733079122975, 'a': 0.001525262154432793, 'l': 0.00019065776930409913}), 'RT': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7fe41f5d82f0>, {' ': 0.9797459031485914, '↺': 0.0012888970723623642, '</s>': 0.0005523844595838704, 'O': 0.0005523844595838704, ':': 0.0007365126127784938, '_': 0.00018412815319462345, '.': 0.0007365126127784938, "'": 0.004050819370281716, '&': 0.00018412815319462345, 'G': 0.00018412815319462345, 'Y': 0.0014730252255569876, 'H': 0.0005523844595838704, '4': 0.00018412815319462345, 'K': 0.00018412815319462345, 'A': 0.0003682563063892469, 'L': 0.0005523844595838704, 'y': 0.00018412815319462345, '…': 0.00018412815319462345, '2': 0.0005523844595838704, '❤': 

defaultdict(<function __main__.lm.<locals>.<lambda>.<locals>.<lambda>>,
            {' ': 0.016648168701442843,
             '.': 0.0011098779134295228,
             '1': 0.0022197558268590455,
             '2': 0.0011098779134295228,
             '</s>': 0.0011098779134295228,
             'C': 0.0011098779134295228,
             'E': 0.0011098779134295228,
             'G': 0.0011098779134295228,
             'I': 0.0011098779134295228,
             'M': 0.0011098779134295228,
             'Q': 0.0011098779134295228,
             'R': 0.0011098779134295228,
             'W': 0.0011098779134295228,
             'a': 0.03551609322974473,
             'b': 0.018867924528301886,
             'c': 0.003329633740288568,
             'd': 0.005549389567147614,
             'e': 0.10432852386237514,
             'h': 0.0011098779134295228,
             'i': 0.058823529411764705,
             'l': 0.2519422863485017,
             'm': 0.0011098779134295228,
             'n': 0.003329633740288

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [0]:
import math

def eval(n, model, data_file_path):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for

  data_file = pd.read_csv(data_file_path)
  count = 0
  total = 0.0
  for data in data_file['tweet_text'].values:
      data = ['<s>'] + list(data) + ['</s>']
      for i in range(len(data) - n + 1):
          count += 1
          word, char = ''.join(data[i:i + n - 1]), data[i + n - 1]
          total += math.log2(model[word][char])

  ent = -1 / count * total
  per = 2 ** ent

  return per

      

In [25]:
model = lm(4, vocabulary, 'en.csv', True)
print(eval(4, model, 'en.csv'))

59.93216357030505


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  file_path = '{}.csv'
  vocabulary = preprocess()

  lang = ['en' ,'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
  df = pd.DataFrame(columns=lang, index=lang)
  for l1 in lang:
    l1_model = lm(n, vocabulary, file_path.format(l1), add_one)
    for l2 in lang:
      df.at[l1, l2] = eval(n, l1_model, file_path.format(l2))

  return df


In [27]:
print (match(2, False))

         en       es       fr       in       it       nl       pt       tl
en  18.2886  30.2626  33.4835  27.5662     27.6  25.3983  37.8465  27.0262
es  29.1031  16.2653  28.9506  30.9691  23.4261  30.0798  26.2961  30.5896
fr  26.1245  27.6946  17.1228  30.1329  24.6027  27.1802  31.9095  31.4539
in  26.6907  34.0945    41.14  18.1508  28.3149  27.0107  41.3162  24.0628
it  28.7876   23.751  30.4275  30.4894  16.7155  30.1302  29.7926  29.8152
nl   25.054  31.7616  29.6374  28.1347  28.8831  17.9473  37.0871  29.2788
pt  29.8409  21.4945  28.7374  32.3514  24.7088  31.2407   16.596  31.7168
tl  24.9965  28.4714  41.4229  23.4979  27.2717  28.1747  35.6522  17.9873


In [28]:
print (match(2, True))

         en       es       fr       in       it       nl       pt       tl
en  21.1437  27.3662  28.6636     29.9  27.6751  27.7564  30.6793  29.6431
es   32.235  18.9488  28.4303  34.4518  24.6186  33.3406  25.0915  34.5761
fr   28.567  25.0542  19.8251  33.0453  26.3247  29.8626  28.0467  35.1086
in  29.7969  29.8572  33.5537   21.294  28.9747  29.9556  33.1538    26.97
it  31.7988  24.3805  29.0279  33.7569  19.4943  33.3721  26.8631   33.451
nl  27.4503  30.5479  30.5624   30.623  30.7544   20.666  33.4526   32.354
pt  33.6295  23.5836  29.4142  36.2009  26.0403   35.171  19.9681   36.174
tl  27.7543  28.4973  33.4273  25.9127  27.8307  31.2726    31.66  21.4386


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [30]:
pd.options.display.max_columns = None
for n in range(0, 4):
  print("\n\nn: {}. add_one: False".format(n + 1))
  print(match(n + 1, False))
  print("\n\nn: {}. add_one: True".format(n + 1))
  print(match(n + 1, True))



n: 1. add_one: False
         en       es       fr       in       it       nl       pt       tl
en  38.5819  41.7284  43.9312  41.7317  41.2459  39.7133  45.2394  45.0282
es  42.1904  36.2134  41.5859  43.8699  40.4491  41.5543  41.0399  47.5528
fr   41.802   40.208  37.5004  44.6866  39.9451  40.9627  42.4084   49.471
in  42.6906  45.4138  48.1969  37.3912  43.8748  41.9243  48.3191  42.9053
it  41.6723  39.0687  40.9179  43.6638  37.5525  41.0965  42.6782  46.7371
nl  40.9808  41.1384  42.1104   41.815  41.1531  37.4997  43.9915  46.7566
pt  42.7496  37.6343  41.0913  43.2408  41.0032  41.7144  37.0763   47.514
tl  42.4023  43.1284   48.603  39.3321  43.3317  42.8451   45.963  40.7645


n: 1. add_one: True
         en       es       fr       in       it       nl       pt       tl
en  38.6302  39.7951  41.1771  41.2136  39.5431  39.4877  41.0814   44.588
es   41.791  36.2645  39.9553  43.2759  39.3731  41.1703  38.0577  47.0437
fr  41.3339  39.3838  37.5459  44.1157   39.524   40.67

# **Good luck!**